# Predicting Next Fixtures

This notebook is dedicated to forecasting the outcomes of upcoming fixtures involving teams that have not yet played their matches. By utilizing the data we have collected and processed in previous notebooks, we can predict the expected goals for each team based on their current statistics and historical performance.

## Objectives of This Notebook

The primary objectives of this notebook include:

1. **Utilizing Current Data**: We will leverage the most recent datasets, which contain updated features such as historical match statistics, team form, and head-to-head (H2H) data, to make informed predictions about upcoming matches.

2. **Predicting Expected Goals**: The focus will be on estimating the expected goals (xG) for each team in the fixtures. This metric is crucial for evaluating a team's attacking and defensive capabilities and provides a clearer picture of potential match outcomes.

3. **Model Application**: We will apply the trained machine learning model developed in previous sections to predict the goals for each team. By inputting the relevant features into the model, we can generate predictions for both home and away teams in each fixture.



In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from Dataset_functions import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
from datetime import date
import warnings
warnings.filterwarnings("ignore")

We begin the prediction process by loading the saved model that was previously trained and optimized. This model will enable us to make accurate predictions regarding the expected goals for upcoming matches.

To ensure the prediction process runs smoothly, the first step involves eliminating any unnecessary variables from our dataset. This step is crucial, as it helps to focus on the relevant features that directly contribute to our goal predictions, thereby enhancing the model's efficiency.

In total, we have 39 matches to predict, which encompass fixtures from the top four European leagues. By concentrating on this specific set of matches, we can apply our model effectively and provide insights into potential outcomes based on the historical and current data of the teams involved.

Overall, this initial stage of loading the model and refining our dataset is vital for setting the groundwork for accurate predictions, enabling us to leverage the model’s capabilities fully.

In [3]:
df = pd.read_csv('default_data_all_variables_v1.csv')
target = pd.read_csv('targets_v1.csv')

df = df.loc[:, ~df.columns.str.contains('HomeTeam|AwayTeam')]
df = df.loc[:, ~df.columns.str.contains('year|month')]
df = df.loc[:, ~df.columns.str.contains('ovr')]
df = df.loc[:, ~df.columns.str.contains('corner')]
df = df.loc[:, ~df.columns.str.contains('TimeBucket')]
df = df.loc[:, ~df.columns.str.contains('Avg')]
data_preds = df.iloc[-39:]

model = load_model("model_fully_optimal.keras")

y_pred = model.predict(data_preds)
res = pd.DataFrame(y_pred, columns = ['FTHG', 'FTAG'])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step 


After predicting the expected goals for each team, the subsequent step involves loading our established criteria for over and under goals. This process is crucial as it helps us identify which matches align with our betting strategies based on the model's predictions.

By applying these conditions, we can categorize the upcoming fixtures into those that meet the criteria for betting on over 2.5 goals and those that fall under the under 2.5 goals category. This classification allows us to focus our betting efforts on matches that have the highest potential for profitability.

Once the categorization is complete, we will compile a comprehensive table that presents our betting propositions. This table will not only include the teams participating in each match but also the predicted goals, the specific over/under conditions, and the corresponding odds from the bookmakers. 

By organizing this information clearly, we empower ourselves to make informed decisions about our betting strategy, ultimately maximizing our chances of success and enhancing our overall profitability.

In [4]:
over_conditions = pd.read_csv('over_conditions.csv')
under_conditions = pd.read_csv('under_conditions.csv')
next_fixtures = pd.read_csv('next_fixture_teams.csv')[-39:]

over_under = pd.read_csv('2_5_goals.csv')[-39:]
over_under.reset_index(inplace = True, drop = True)

next_fixtures = pd.merge(next_fixtures[['HomeTeam',	'AwayTeam']], over_under, on =['HomeTeam',	'AwayTeam'], how = 'left')
next_fixtures[['home_goals_pred', 'away_goals_pred']] = y_pred
next_fixtures['total_goals'] = next_fixtures['home_goals_pred'] + next_fixtures['away_goals_pred']

all_considered_matches_over = pd.DataFrame(columns = next_fixtures.columns)
for index, row in over_conditions.iterrows():
    matches_to_add = next_fixtures.loc[(next_fixtures['total_goals']>row['goals_total_over']) & (next_fixtures['Avg>2.5']>row['over_2_5_val'])]
    all_considered_matches_over = pd.concat([all_considered_matches_over, matches_to_add])
all_considered_matches_over.drop_duplicates(inplace= True)
all_considered_matches_over['over_under'] = ['over']*len(all_considered_matches_over)

all_considered_matches_under = pd.DataFrame(columns = next_fixtures.columns)
for index, row in under_conditions.iterrows():
    matches_to_add = next_fixtures.loc[(next_fixtures['total_goals']<row['goals_total_under']) & (next_fixtures['Avg<2.5']>row['under_2_5_val'])]
    all_considered_matches_under = pd.concat([all_considered_matches_under, matches_to_add])
all_considered_matches_under.drop_duplicates(inplace= True)
all_considered_matches_under['over_under'] = ['under']*len(all_considered_matches_under)


today = date.today()
betting_summary = pd.concat([all_considered_matches_over, all_considered_matches_under])
betting_summary.to_csv(f'betting_summary_{str(today)}.csv', index=False)

An optimal feature of this project would be to automate the process of sending the compiled betting propositions table via email. By sharing this valuable information with others, we can disseminate our betting strategies and insights effectively. This not only enhances the collaborative aspect of our project but also opens up opportunities for monetization.

By establishing a network of interested individuals who receive these betting propositions, we could potentially create a subscription model or offer premium services that provide exclusive insights and analysis. This could generate additional revenue streams while allowing others to benefit from our data-driven approach to betting.

Ultimately, by leveraging the predictive capabilities of our model and sharing the results with a broader audience, we can amplify our impact and maximize the profitability of our project, turning a passionate endeavor into a viable business opportunity.

In [31]:
send_email(betting_summary, "ciesliktymek@gmail.com", today)

Email sent to ciesliktymek@gmail.com


### Exemplary table with betting tips. 

In [5]:
betting_summary

,HomeTeam,AwayTeam,Avg>2.5,Avg<2.5,home_goals_pred,away_goals_pred,total_goals,over_under
17,M'gladbach,Union Berlin,1.77,1.98,1.732872,1.130997,2.863869,over
20,Genoa,Juventus,2.26,1.59,1.005048,1.917117,2.922165,over
21,Bologna,Atalanta,1.97,1.78,1.493544,1.400697,2.894241,over
35,Ath Madrid,Real Madrid,1.75,2.00,1.452736,1.588643,3.041379,over
14,Mainz,Heidenheim,1.70,2.07,1.383921,1.666342,3.050263,over
19,Udinese,Inter,1.71,2.06,1.000063,2.111656,3.111719,over
24,Ipswich,Aston Villa,1.71,2.07,1.123872,2.266101,3.389973,over
1,Milan,Lecce,1.47,2.55,1.582038,0.706984,2.289022,under
37,Parma,Cagliari,1.72,2.05,1.133076,1.153461,2.286537,under


In summary, we have successfully developed a model to predict goals for the upcoming matches. By leveraging the over/under conditions, we can strategically identify fixtures that maximize potential yield. By placing bets on these selected matches, we position ourselves to generate a passive profit.

This approach not only relies on our data-driven predictions but also incorporates established betting strategies that have the potential to yield favorable outcomes. As we continue to refine our model and enhance our betting tactics, we can further optimize our chances for consistent profitability in the sports betting landscape.